In [66]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np

from mltu.inferenceModel import OnnxInferenceModel
from mltu.utils.text_utils import ctc_decoder, get_cer
from tqdm import tqdm

### Carregar o dataset

In [55]:
df = pd.read_csv('../data/data_20230622/data.csv', sep=';')

In [56]:
df.head()

,ID,Label,UserId,UserName,UserRole,Request,SheetId,SheetName,Date,Parent1Name,Parent2Name,Parent3Name
0,0,47,9b705e6cd6,Luis,PARTICIPANT,5f6ae87ce45a8d000156814a,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN
1,1,26,0f1a3956d7,Matilde,PARTICIPANT,5f6ae88be45a8d000156814c,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN
2,2,40,ea5dee75ac,Lorenzo,PARTICIPANT,5f6ae8a3e45a8d000156814e,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN
3,3,48,ea5dee75ac,Lorenzo,PARTICIPANT,5f6ae8a3e45a8d000156814e,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN
4,4,41,ea5dee75ac,Lorenzo,PARTICIPANT,5f6ae8a3e45a8d000156814e,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN


### Tratamento do dataset

In [57]:
# criar um novo dataframe com as colunas id e label
df = df[['ID', 'Label']]

In [58]:
# transformar a coluna id em string
df['ID'] = df['ID'].astype(str)

In [59]:
# adicionar uma string '../data/data_20230622/' no início da coluna id
df['ID'] = '../data/data_20230622/' + df['ID'] + '.jpeg'

In [60]:
# guardar esse novo dataframe em um arquivo csv
if not os.path.exists('labels.csv'):    
    df.to_csv('labels.csv', index=False)
else :
    print('Arquivo já existe')    

Arquivo já existe


### Rodar o modelo

In [64]:
class ImageToWordModel(OnnxInferenceModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def predict(self, image: np.ndarray):
        image = cv2.resize(image, self.input_shape[:2][::-1])

        image_pred = np.expand_dims(image, axis=0).astype(np.float32)

        preds = self.model.run(None, {self.input_name: image_pred})[0]

        text = ctc_decoder(preds, self.vocab)[0]

        return text

In [65]:
model = ImageToWordModel("../handwriting_recognition_torch/Models/08_handwriting_recognition_torch/202306131539/model.onnx")

In [ ]:
accum_cer = []
for i in tqdm(range(len(df))):
    image = cv2.imread(df['ID'][i])
    text = model.predict(image)
    cer, distance = get_cer(df['Label'][i], text)
    print(f'Label: {df["Label"][i]} - Text: {text} - CER: {cer} - Distance: {distance}')
    accum_cer.append(cer)
print(f'Mean CER: {np.mean(accum_cer)}')    